In [3]:
import sys
import os

# Add the current working directory (where spt_utils.py is located) to the Python path
sys.path.append(os.getcwd())

In [4]:
from spt_utils import floquet_spt_step, generate_disorder, initialize_state

ModuleNotFoundError: No module named 'spt_utils'

In [2]:
# spt_utils.py
# Utility functions for Floquet SPT-MBL simulation

from qiskit import QuantumCircuit
import numpy as np

def floquet_spt_step(n_qubits, h_fields, J):
    """
    Construct a QuantumCircuit implementing one Floquet cycle:
    U_F = U2 * U1

    - U1 = exp(-i sum h_j Z_j)     [disorder, local fields]
    - U2 = exp(-i J sum X_{j-1} Z_j X_{j+1}) [entangling cluster term]

    Parameters:
    - n_qubits: int — number of qubits (≥3)
    - h_fields: array of float — random fields h_j for each site
    - J: float — coupling strength for XZX interaction

    Returns:
    - QuantumCircuit implementing one Floquet step
    """
    qc = QuantumCircuit(n_qubits)

    # Disordered Z-field layer (U1)
    for j in range(n_qubits):
        qc.rz(2 * h_fields[j], j)  # RZ(2h) = exp(-i h Z)

    # XZX interaction layer (U2)
    for j in range(1, n_qubits - 1):
        qc.cx(j - 1, j)
        qc.cx(j + 1, j)
        qc.rz(2 * J, j)
        qc.cx(j + 1, j)
        qc.cx(j - 1, j)

    return qc

def generate_disorder(n_qubits, strength=0.5, seed=None):
    """
    Generate a random disorder field h_j ∈ [-strength, +strength] for each qubit
    
    Parameters:
    - n_qubits: int
    - strength: float
    - seed: int or None

    Returns:
    - numpy array of length n_qubits
    """
    if seed is not None:
        np.random.seed(seed)
    return np.random.uniform(-strength, strength, n_qubits)

def initialize_state(n_qubits, config='edge_plus'):
    """
    Create an initial statevector based on the desired configuration.

    Options:
    - 'plus': all |+⟩
    - 'edge_polarized': Z-polarized edge, X-polarized bulk

    Returns:
    - Statevector object
    """
    from qiskit.quantum_info import Statevector

    qc = QuantumCircuit(n_qubits)
    if config == 'plus':
        qc.h(range(n_qubits))
    elif config == 'edge_polarized':
        qc.x(0)  # |1⟩ on edge
        qc.h(range(1, n_qubits))
    else:
        raise ValueError("Unknown initial state config")

    return Statevector.from_instruction(qc)
